In [1]:
#!pip install ollama
import ollama
import re
from IPython.display import Markdown, display

In [2]:
samples = [
        {
            "content": "DeepSeek R1 is a reasoning-focused large language model (LLM) that leverages pure reinforcement learning to develop advanced reasoning abilities. It's optimized for complex reasoning tasks and delivers exceptional performance on logic, mathematics, and reasoning benchmarks while maintaining strong general capabilities.",
            "metadata": {
                "name": "DeepSeek R1 Documentation",
                "path": "/data/documents/deepseek_r1_docs.pdf"
            }
        },
        {
            "content": "Retrieval-Augmented Generation (RAG) enhances LLM responses by retrieving relevant context before generating answers. This approach grounds responses in factual information, reducing hallucinations. When using DeepSeek R1 for RAG applications, leverage R1's reasoning capabilities in the generation stage, not for retrieval.",
            "metadata": {
                "name": "RAG Best Practices Guide",
                "path": "/data/knowledge/rag_best_practices.md"
            }
        }
    ]

In [3]:
def source_summarizer(context_documents, llm_model="deepseek-r1"):
    system_message = """
    You are an expert summarizer working within a RAG system. Your task is to create a concise, accurate summary of the provided information while properly attributing all facts to their sources.

    Guidelines:
    - Create a clear, coherent summary limited to 3-5 sentences
    - Focus on the most important facts and insights
    - Maintain factual accuracy without adding new information
    - Use neutral, professional language
    - Cite EVERY piece of information using the format [Document Name](document_path)
    - Place citations immediately after the relevant information
    - Ensure each citation is correctly matched to its source
    - Return only the plain text summary without markdown formatting
    """

    formatted_context = "\n".join(
        f"Content: {doc['content']}\nSource: {doc['metadata']['name']}\nPath: {doc['metadata']['path']}"
        for doc in context_documents
    )

    prompt = f"""
    Here are the documents to summarize:
    
    {formatted_context}
    
    Provide a concise summary with proper citations:
    """
    
    response = ollama.chat(
        model=llm_model,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt}
        ]
    )
    
    response_content = response["message"]["content"]
    
    # Clean markdown formatting if present
    try:
        final_content = re.sub(r"<think>.*?</think>", "", response_content, flags=re.DOTALL).strip()
    except:
        final_content = response_content.strip()

    # Extract metadata from all documents
    document_names = [doc['metadata']['name'] for doc in context_documents]
    document_paths = [doc['metadata']['path'] for doc in context_documents]

    return {
        "content": final_content,
        "metadata": {
            "name": document_names,
            "path": document_paths
        }
    }

In [4]:
smry = source_summarizer(samples)
smry

{'content': 'DeepSeek R1 is a reasoning-focused large language model (LLM) developed with pure reinforcement learning, designed to excel in complex tasks like logic and mathematics while maintaining strong general capabilities. Retrieval-Augmented Generation (RAG) enhances LLM responses by incorporating relevant context before generating answers, ensuring more factual results. When using DeepSeek R1 for RAG applications, focus on leveraging its reasoning capabilities during the generation stage rather than employing it solely for retrieval tasks. [DeepSeek R1 Documentation](/data/documents/deepseek_r1_docs.pdf) and [RAG Best Practices Guide](/data/knowledge/rag_best_practices.md).',
 'metadata': {'name': ['DeepSeek R1 Documentation',
   'RAG Best Practices Guide'],
  'path': ['/data/documents/deepseek_r1_docs.pdf',
   '/data/knowledge/rag_best_practices.md']}}

In [5]:
display(Markdown(smry['content']))

DeepSeek R1 is a reasoning-focused large language model (LLM) developed with pure reinforcement learning, designed to excel in complex tasks like logic and mathematics while maintaining strong general capabilities. Retrieval-Augmented Generation (RAG) enhances LLM responses by incorporating relevant context before generating answers, ensuring more factual results. When using DeepSeek R1 for RAG applications, focus on leveraging its reasoning capabilities during the generation stage rather than employing it solely for retrieval tasks. [DeepSeek R1 Documentation](/data/documents/deepseek_r1_docs.pdf) and [RAG Best Practices Guide](/data/knowledge/rag_best_practices.md).

In [6]:
smry2 = source_summarizer(samples, llm_model='llama3.2')
display(Markdown(smry2['content']))

DeepSeek R1 is a reasoning-focused large language model that leverages reinforcement learning to develop advanced reasoning abilities. It excels in logic, mathematics, and reasoning benchmarks while maintaining strong general capabilities [DeepSeek R1 Documentation] (/data/documents/deepseek_r1_docs.pdf). In Retrieval-Augmented Generation (RAG) applications, DeepSeek R1's reasoning capabilities should be used for generation, rather than retrieval, to ground responses in factual information and reduce hallucinations [RAG Best Practices Guide] (/data/knowledge/rag_best_practices.md).